# Assignment 6
## Group Members:
* ### Nils Dunlop, e-mail: gusdunlni@student.gu.se
* ### Francisco Alejandro Erazo Piza, e-mail: guserafr@student.gu.se

## Problem 1

In [7]:
import sys
from collections import deque

def dijkstra(graph, src):
    inf = sys.maxsize
    node_data = {node: {'cost': inf, 'pred': []} for node in graph}
    node_data[src]['cost'] = 0

    priority_queue = [(0, src)]

    while priority_queue:
        priority_queue.sort(reverse=True, key=lambda x: x[0])

        current_cost, current_node = priority_queue.pop()

        for neighbor, time in graph[current_node].items():
            cost = node_data[current_node]['cost'] + time
            if cost < node_data[neighbor]['cost']:
                node_data[neighbor]['cost'] = cost
                node_data[neighbor]['pred'] = node_data[current_node]['pred'] + [current_node]
                priority_queue.append((cost, neighbor))

    # for node, data in node_data.items():
    #     print(f"Node: {node}")
    #     print(f"Shortest Distance from {src}: {data['cost']}")
    #     print(f"Shortest Path from {src}: {data['pred']} -> {node}\n")

    farthest_node = max(node_data, key=lambda node: node_data[node]['cost'])
    print(f"Shortest Distance to All nodes: {node_data[farthest_node]['cost']}")
    print(f"Shortest Path from {src}: {node_data[farthest_node]['pred'] + [farthest_node]}")


def dag_shortest_path(graph, src):
    inf = sys.maxsize
    node_data = {node: {'cost': inf, 'pred': []} for node in graph}
    node_data[src]['cost'] = 0

    def topological_sort(graph):
        sorted_nodes = deque()
        visited = set()

        def dfs(node):
            visited.add(node)
            for neighbor in graph[node]:
                if neighbor is not visited:
                    dfs(neighbor)
            sorted_nodes.appendleft(node)

        for node in graph:
            if node not in visited:
                dfs(node)

        return sorted_nodes

    sorted_nodes = topological_sort(graph)

    for node in sorted_nodes:
        for neighbor, time in graph[node].items():
            cost = node_data[node]['cost'] + time
            if cost < node_data[neighbor]['cost']:
                node_data[neighbor]['cost'] = cost
                node_data[neighbor]['pred'] = node_data[node]['pred'] + [node]

    # for node, data in node_data.items():
    #     print(f"Node: {node}")
    #     print(f"Shortest Distance from {src}: {data['cost']}")
    #     print(f"Shortest Path from {src}: {data['pred']} -> {node}\n")

    farthest_node = max(node_data, key=lambda node: node_data[node]['cost'])
    print(f"Shortest Distance to All nodes: {node_data[farthest_node]['cost']}")
    print(f"Shortest Path from {src}: {node_data[farthest_node]['pred'] + [farthest_node]}")


if __name__ == "__main__":
    dijkstra_graph = {
        'A':{'B':2,'C':4},
        'B':{'A':2,'C':3, 'D':8},
        'C':{'A':4,'B':3, 'E':5, 'D':2},
        'D':{'B':8,'C':2, 'E':11, 'F':22},
        'E':{'C':5,'D':11, 'F':1},
        'F':{'D':22,'E':1}
    }

    dag_graph = {
        'A': {'B': 3, 'C': 2},
        'B': {'C': 7, 'D': 1},
        'C': {'D': 5},
        'D': {},
    }

    source = 'A'
    dijkstra(dijkstra_graph, source)
    dag_shortest_path(dag_graph, source)

Shortest Distance to All nodes: 10
Shortest Path from A: ['A', 'C', 'E', 'F']
deque(['A', 'C', 'D', 'B', 'D', 'C', 'D'])
deque(['A', 'C', 'D', 'B', 'D', 'C', 'D'])
deque(['A', 'C', 'D', 'B', 'D', 'C', 'D'])
deque(['A', 'C', 'D', 'B', 'D', 'C', 'D'])
Shortest Distance to All nodes: 4
Shortest Path from A: ['A', 'B', 'D']


## Problem 2

In [9]:
import tarfile

def extract_tar_to_dict(tar_file_path):
    extracted_files = {}

    try:
        with tarfile.open(tar_file_path, 'r:gz') as tar:
            for item in tar:
                if item.name.startswith('Data/._'):
                    continue
                if item.isfile() and 'tram' in item.name and item.name.endswith('.txt'):
                    file = tar.extractfile(item)
                    if file:
                        tram_dict = {}

                        for line in file:
                            line = line.decode('utf-8', errors='ignore').strip()

                            tram_stop, time = line.split(', ')

                            tram_dict[tram_stop] = int(time)

                        tram_number = item.name.replace('Data/', '').replace('.txt', '')
                        extracted_files[tram_number] = tram_dict
    except FileNotFoundError:
        print(f"{tar_file_path} not found.")
    except tarfile.ReadError:
        print(f"{tar_file_path} is not a tar file.")

    return extracted_files


tar_file_path = 'Data_A6.tar.gz'
extracted_dict = extract_tar_to_dict(tar_file_path)
print(extracted_dict)

{'Axel Dahlströms Torg': 1, 'Marklandsgatan': 3, 'Botaniska Trädgården': 2, 'Linnéplatsen': 1, 'Olivedalsgatan': 2, 'Seminariegatan': 1, 'Brunnsgatan': 1, 'Handelshögskolan': 2, 'Vasa Viktoriagatan': 1, 'Vasaplatsen': 1, 'Grönsakstorget': 1, 'Domkyrkan': 3, 'Brunnsparken': 2, 'Centralstationen': 3, 'Ullevi Södra': 2, 'Scandinavium': 1, 'Korsvägen': 2, 'Liseberg Södra': 2, 'Almedal': 1, 'Elisedal': 1, 'Varbergsgatan': 1, 'Lana': 1, 'Krokslätts torg': 1, 'Krokslätts Fabriker': 2, 'Lackarebäck': 1, 'Mölndals sjukhus': 2, 'Mölndals Innerstad': 0}


In [66]:
def get_complete_tram_data(extracted_dict):
    reverse_dict = {}

    for tram, stops in extracted_dict.items():
        reversed_stops = {}
        prev_stop = None
        prev_time = None

        for stop, time in list(stops.items())[::-1]:
            if prev_stop is not None:
                reversed_stops[prev_stop] = prev_time

            prev_stop = stop
            prev_time = time

        reversed_stops[prev_stop] = 0
        reverse_tram = f"{tram}_reverse"
        reverse_dict[reverse_tram] = reversed_stops

    return {**extracted_dict, **reverse_dict}


def get_tram_hubs(complete_tram_data, extracted_dict):
    all_tram_stops, tram_hubs = set(), set()
    connections_count = {}

    for inner_dict in complete_tram_data.values():
        for key in inner_dict.keys():
            all_tram_stops.add(key)

    for tram_stop in all_tram_stops:
        connections_count[tram_stop] = 0

    for tram_line in extracted_dict.values():  
        stops = list(tram_line.keys())
        for i in range(len(stops) - 1):
            current_stop = stops[i]
            next_stop = stops[i + 1]
            connections_count[current_stop] += 1
            connections_count[next_stop] += 1

    for stop, count in connections_count.items():
        if count >= 3:
            tram_hubs.add(stop)

    return tram_hubs


# Using the functions:
extracted_dict = extract_tar_to_dict('Data_A6.tar.gz')
complete_tram_data = get_complete_tram_data(extracted_dict)
tram_hubs = get_tram_hubs(complete_tram_data, extracted_dict)

print(tram_hubs)

{'Hagakyrkan', 'Eketrägatan', 'Bellevue', 'Beväringsgatan', 'Mariaplan', 'Kaggeledstorget', 'Masthuggstorget', 'SKF', 'Friskväderstorget', 'Ättehögsgatan', 'Galileis gata', 'Brunnsparken', 'Nya Varvsallén', 'Temperaturgatan', 'Vagnhallen Majorna', 'Frölunda Torg Spårvagn', 'Olskrokstorget', 'Hjällbo', 'Domkyrkan', 'Kungssten', 'Roddföreningen', 'Januarigatan', 'Hagen', 'Chalmers', 'Kaptensgatan', 'Käringberget', 'Tingvallsvägen', 'Chapmans Torg', 'Gropegårdsgatan', 'Gamlestads Torg', 'Grönsakstorget', 'Varbergsgatan', 'Solrosgatan', 'Frihamnen', 'Ejdergatan', 'Centralstationen', 'Sälöfjordsgatan', 'Wavrinskys plats', 'Vasa Viktoriagatan', 'Sahlgrenska Huvudentré', 'Långedrag', 'Lilla Bommen', 'Stenpiren', 'Väderilsgatan', 'Runstavsgatan', 'Sanatoriegatan', 'Sannaplan', 'Mildvädersgatan', 'Nymilsgatan', 'Kviberg', 'Lackarebäck', 'Berzeliigatan', 'Stockholmsgatan', 'Ullevi Södra', 'Nymånegatan', 'Mölndals sjukhus', 'Prinsgatan', 'Munkebäckstorget', 'Botaniska Trädgården', 'Axel Dahlström